`--- Day 5: Alchemical Reduction ---`

In [1]:
import unittest
import re

In [2]:
polymer = open('input.txt').read().strip()
print(f'polymer length is {len(polymer)}\npolymer = {polymer[:10]}...{polymer[-10:]}')

polymer length is 50000
polymer = bBkKQqgaAG...FJnZNnwxXW


# Construct tests

For example:
```
    In aA, a and A react, leaving nothing behind.
    In abBA, bB destroys itself, leaving aA. As above, this then destroys itself, leaving nothing.
    In abAB, no two adjacent units are of the same type, and so nothing happens.
    In aabAAB, even though aa and AA are of the same type, their polarities match, and so nothing happens.
```
Now, consider a larger example, dabAcCaCBAcCcaDA:
```
dabAcCaCBAcCcaDA  The first 'cC' is removed.
dabAaCBAcCcaDA    This creates 'Aa', which is removed.
dabCBAcCcaDA      Either 'cC' or 'Cc' are removed (the result is the same).
dabCBAcaDA        No further actions can be taken.
```
After all possible reactions, the resulting polymer contains 10 units.

In [3]:
testdata = {              'aA': '',
                        'abBA': '',
                        'abAB': 'abAB',
                      'aabAAB': 'aabAAB',
            'dabAcCaCBAcCcaDA': 'dabCBAcaDA'}

testdata2 = {'dabAcCaCBAcCcaDA': 'dabAaCBAcCcaDA',
               'dabAaCBAcCcaDA': 'dabCBAcCcaDA',
                 'dabCBAcCcaDA': 'dabCBAcaDA',
                   'dabCBAcaDA': 'dabCBAcaDA'}

testdata3 = {'dabAcCaCBAcCcaDA': 'dabAaCBAcaDA',
               'dabAaCBAcCcaDA': 'dabCBAcaDA',
                   'dabCBAcaDA': 'dabCBAcaDA'}

In [4]:
class TestIt(unittest.TestCase):
    # part 1
    def test_poly_reduce(self):
        for reducer in reducers:
            for test, res in testdata.items():
                with self.subTest(poly=test, reducer=reducer):
                    self.assertEqual(poly_reduce(test, reducer=reducer), res)
                
    def test_reduce_first(self):
        for test, res in testdata2.items():
            with self.subTest(poly=test):
                self.assertEqual(reduce_first(test), res)
                
    def test_reduce_all(self):
        for test, res in testdata3.items():
            with self.subTest(poly=test):
                self.assertEqual(reduce_all(test), res)
        

# Implement part 1 solution

In [5]:
rel = []
for c in 'abcdefghijklmnopqrstuvwxyz':
    rel.append(c + c.upper())
    rel.append(c.upper() + c)

reactive_units_re = re.compile('|'.join(rel))

def reduce_first(poly):
    return re.sub(reactive_units_re, '', poly, count=1)

def reduce_all(poly):
    return re.sub(reactive_units_re, '', poly)

def reduce_sequential(poly):
    for r in rel:
        poly = re.sub(r, '', poly)
    return poly

reducers = [reduce_first, reduce_all, reduce_sequential]

def poly_reduce(poly, reducer=reduce_sequential):
    last_length = len(poly) + 1 # start out with it not equal to len(poly)
    while last_length != len(poly):
        last_length = len(poly)
        poly = reducer(poly)
    return poly

# Run part 1 tests

In [6]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestIt)
unittest.TextTestRunner(verbosity=2).run(suite)

test_poly_reduce (__main__.TestIt) ... ok
test_reduce_all (__main__.TestIt) ... ok
test_reduce_first (__main__.TestIt) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.011s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

# Performance checks

In [7]:
test_poly = polymer[:10000]

In [8]:
%%timeit
res = poly_reduce(test_poly, reducer=reduce_sequential)

3.79 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
res = poly_reduce(test_poly, reducer=reduce_all)

9.3 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
res = poly_reduce(test_poly, reducer=reduce_first)

706 ms ± 42.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Part 1 answer

In [11]:
%%timeit -n 1 -r 1
print(f'Part 1: {len(poly_reduce(polymer))}')

Part 1: 10598
450 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Part 2

In [12]:
%%timeit -n 1 -r 1
reduced_lengths = []
for c in 'abcdefghijklmnopqrstuvwxyz':
    r = f'[{c}{c.upper()}]' # '[aA]' etc.
    reduced_lengths.append(len(poly_reduce(re.sub(r ,'', polymer))))
print(f'Part 2: {min(reduced_lengths)}')

Part 2: 5312
12.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
